In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
import random
import dataclasses
import jax
import optax

import jax.numpy as jnp
import numpy as np

from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from penzai.data_effects.side_output import SideOutputValue
from micrlhf.utils.activation_manipulation import add_vector
from micrlhf.utils.load_sae import get_sae
from functools import partial

In [3]:
filename = "models/phi-3-16.gguf"
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="tpu:0")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
from task_vector_utils import load_tasks, ICLDataset, ICLSequence
tasks = load_tasks()

Cloning into 'itv'...
fatal: unable to access 'https://github.com/roeehendel/icl_task_vectors data/itv/': URL using bad/illegal format or missing URL


In [6]:
from micrlhf.llama import LlamaBlock
from micrlhf.sampling import sample, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)

In [7]:
def tokenized_to_inputs(input_ids, attention_mask):
    token_array = jnp.asarray(input_ids)
    token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")

    mask_array = jnp.asarray(attention_mask, dtype=jnp.bool)
    mask_array = jax.device_put(mask_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    mask_array = pz.nx.wrap(mask_array, "batch", "seq").untag("batch").tag("batch")

    inputs = llama.inputs.from_basic_segments(token_array)
    return inputs

In [8]:
prompt = "<user>Follow the pattern\n{}"

In [9]:

target_layer = 17

In [10]:
task_names = [
    "en_es"
]

task = "en_es"

n_seeds = 10

# n_few_shots, batch_size, max_seq_len = 64, 64, 512
n_few_shots, batch_size, max_seq_len = 32, 64, 256

In [11]:
def get_logprob_diff(logits: jnp.ndarray, target_tokens, print_results=False):
    logprobs = jax.nn.log_softmax(logits, axis=-1)
    answer_logprobs = logprobs[:, -1]

    target_logprobs = jnp.take_along_axis(answer_logprobs, target_tokens[:, None], axis=-1).squeeze()

    if print_results:
        print(
            tokenizer.decode(answer_logprobs.argmax(axis=-1))
        )

        print(
            tokenizer.decode(target_tokens)
        )

    return target_logprobs


In [12]:
from micrlhf.llama import LlamaBlock
from functools import partial

def make_get_resids(llama, layer_target):
    get_resids = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer_target
                                                                              ).apply(lambda x:
        pz.nn.Sequential([
            pz.de.TellIntermediate.from_config(tag=f"resid_pre"),
            x
        ])
    )
    get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
    return get_resids

In [13]:
import dataclasses
def get_loss(weights, dictionary, inputs, target_tokens, taker, initial_resids):
    weights = jax.nn.relu(weights)
    

    recon = jnp.einsum("fv,f->v", dictionary, weights)
    recon = recon.astype('bfloat16')

    modified = pz.nx.nmap(lambda a, b: a.at[-1].add(b))(
        initial_resids.untag("seq", "embedding"), pz.nx.wrap(recon, "embedding").untag("embedding")
        ).tag("seq", "embedding")

    inputs = dataclasses.replace(inputs, tokens=modified)

    logits = taker(inputs).unwrap("batch", "seq", "vocabulary")

    logprob_diff = get_logprob_diff(logits, target_tokens)
    loss = -logprob_diff.mean()

    return loss + 1e-2 * jnp.linalg.norm(weights, ord=1), ((weights != 0).sum(), loss)
    # return loss, ((weights != 0).sum(), loss)

In [14]:
def train_step(weights, opt_state, dictionary, inputs, target_tokens, taker, initial_resids, optimizer, lwg, pos_only=True):
    (loss, (l0, loss_)), grad = lwg(weights, dictionary, inputs, target_tokens, taker, initial_resids)

    updates, opt_state = optimizer.update(grad, opt_state, weights)
    weights = optax.apply_updates(weights, updates)
    # weights_abs = jnp.abs(weights)
    # weights = jnp.sign(weights) * jax.nn.relu(weights_abs - shrinkage)

    return loss, weights, opt_state, dict(l0=l0, loss=loss_)

In [15]:
jittify = lambda x: partial(jax.jit(lambda lr, *args, **kwargs: lr(*args, **kwargs)[1][0].value), x)

In [37]:
layer_source = 17
get_resids_initial = make_get_resids(llama, layer_source)
get_resids_initial = jittify(get_resids_initial)

In [38]:
pairs = tasks[task]
pairs = [list(x) for x in pairs.items()]
seed = 0

dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=seed+1)


tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
inputs = tokenized_to_inputs(
    **tokenized
)

target_tokens = [x[1] for x in tokenizer(dataset.completions)["input_ids"]]
target_tokens = jnp.asarray(target_tokens)

In [39]:
initial_resids = get_resids_initial(inputs)

In [40]:
layer_target = target_layer

taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
    lambda i, x: x if i >= layer_target else pz.nn.Identity()
).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

In [27]:

sae = get_sae(18, 6)
dictionary = sae["W_dec"]
dictionary.shape

(49152, 3072)

In [42]:


optimizer = optax.chain(
    optax.adam(3e-2),
    optax.zero_nans(),
    
)

lwg = jax.value_and_grad(get_loss, has_aux=True)
shrinkage = 0

# @partial(jax.jit, donate_argnums=(0, 1))

In [43]:
weights = jnp.ones(dictionary.shape[0]) * 0.1

In [44]:
_, grad = lwg(weights, dictionary, inputs, target_tokens, taker, initial_resids)

In [45]:
jax.lax.top_k(
    grad, 20
)

[# jax.Array float32(20,) ≈0.09 ±0.028 [≥0.061, ≤0.18] nonzero:20
   Array([0.17584193, 0.13643602, 0.12201   , 0.10865737, 0.10761713,
          0.10272211, 0.09796266, 0.09370612, 0.08502723, 0.08187935,
          0.07901955, 0.07308553, 0.07174832, 0.07121938, 0.0695884 ,
          0.06793158, 0.06730887, 0.0646219 , 0.06300792, 0.06144137],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥848, ≤48_313] nonzero:20
   Array([41407, 18727, 26461, 47591,  2215, 48313, 17537, 35733, 43501,
          46642,  3335, 43265, 46985, 22130, 29927,  5694,  4621, 46339,
            848, 30692], dtype=int32)
 ]

In [ ]:
iterations = 2000

weights = jnp.ones(dictionary.shape[0]) * 0.1
opt_state = optimizer.init(weights)

min_loss = 1e9
early_stopping_steps = 100
early_stopping_counter = 0


for _ in (bar := trange(iterations)):
    loss, weights, opt_state, metrics = train_step(weights, opt_state, dictionary, inputs, target_tokens)

    if metrics["loss"] < min_loss:
        min_loss = metrics["loss"]
        early_stopping_counter = 0
    

    tk = jax.lax.top_k(weights, 2)

    bar.set_postfix(loss_optim=loss, **metrics, top=tk[1][0], top_diff=(tk[0][0] - tk[0][1]) / tk[0][0])

    early_stopping_counter += 1
    if early_stopping_counter > early_stopping_steps:
        break

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
_, i = jax.lax.top_k(jnp.abs(weights), 20)
weights[i] / jnp.abs(weights).max(), i

(# jax.Array float32(20,) ≈0.16 ±0.32 [≥-0.031, ≤1.0] nonzero:20
   Array([ 1.        ,  0.8120634 ,  0.6285117 ,  0.62683046,  0.27645394,
           0.08325835,  0.03821881, -0.03123521, -0.03082848, -0.02749531,
          -0.02738206, -0.02709624, -0.02674427, -0.02654962, -0.02622802,
          -0.02615973, -0.02609207, -0.0260878 , -0.025595  , -0.02521051],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥2_719, ≤48_025] nonzero:20
   Array([10018, 20311, 40023,  5145, 45145,  2719, 15415, 24802, 20375,
          19505, 38618, 36786, 47611, 31726, 12632, 10615, 47480, 48025,
           3048, 40850], dtype=int32)
 )

In [ ]:
_, i = jax.lax.top_k(jnp.abs(weights), 20)
weights[i] / jnp.abs(weights).max(), i

In [ ]:
_, i = jax.lax.top_k(jnp.abs(weights), 20)
weights[i] / jnp.abs(weights).max(), i

In [23]:
def check_feature(feature: int, task:str, target_layer, sae, print_results=False):
    pairs = tasks[task]
    pairs = [list(x) for x in pairs.items()]
    dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=10)


    tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
    inputs = tokenized_to_inputs(
        **tokenized
    )

    target_tokens = [x[1] for x in tokenizer(dataset.completions)["input_ids"]]
    target_tokens = jnp.asarray(target_tokens)

    # weights, recon = grad_pursuit(tv * 2, sae["W_dec"], k, pos_only=True)

    recon = sae["W_dec"][feature] * 20
    recon = recon.astype('bfloat16')

    act_add = add_vector(
        llama, recon, target_layer, scale=1.0, position="last"
    )

    logits = act_add(inputs).unwrap("batch", "seq", "vocabulary")

    # print(logits[:, -1].mean(axis=-1))

    logprobs = jax.nn.log_softmax(logits, axis=-1)
    answer_logprobs = logprobs[:, -1]

    target_logprobs = jnp.take_along_axis(answer_logprobs, target_tokens[:, None], axis=-1).squeeze()

    if print_results:
        print(
            tokenizer.decode(answer_logprobs.argmax(axis=-1))
        )

        print(
            tokenizer.decode(target_tokens)
        )

    return (target_logprobs - answer_logprobs.max(axis=-1))

In [28]:
check_feature(16245, "en_ru", 18, sae, print_results=True)

cand br amb rice trees por bur answer calendar book bread money wind not vac boat be aut kitchen ice less ice ph second bat season math bat vac viol mount bi bus fl food be trees table me time drink product dan so chair sell tennis math season minutes follow business bi winter p amb season pie check product * vac bus price
кон зав ско ри дере сви га вопро ка кни х день ок те команди ло г ос ку мо у мо а се ко сез мате ко от с го би авто ф пи г дере сто е да на проду та су сту прода т мате сез ми учи би би з ру ско сез пи реги проду з от авто це


# jax.Array bfloat16(64,) ≈-1.3e+01 ±1.9 [≥-1.8e+01, ≤-8.8] nonzero:64
  Array([-13.3125, -11.875, -12.5625, -11.3125, -14.0625, -17, -18, -14.25,
         -13.375, -11, -14.1875, -12.9375, -12.25, -17.375, -14.5, -14.4375,
         -14.625, -12.6875, -12.75, -13.0625, -12.25, -13.0625, -12.0625,
         -13.125, -12.75, -14.375, -14.8125, -12.75, -11.0625, -13.375,
         -14.625, -10.875, -16.5, -12.9375, -13.0625, -14.625, -14.0625,
         -12.9375, -11.5, -10.25, -11.3125, -12.25, -14, -8.75, -16.25,
         -11.625, -12, -14.8125, -14.375, -10.375, -13.5, -11.4375, -10.875,
         -12, -17.25, -12.5625, -14.375, -11.6875, -13.0625, -12.25, -12.25,
         -11.0625, -16.5, -11.5625], dtype=bfloat16)

In [ ]:
for feature in i:
    print(
        f"Feature {feature}: {check_feature(feature)}"
    )

# get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), target_tokens, print_results=True)

Feature 10018: -7.5
Feature 20311: -2.375
Feature 40023: -4.5
Feature 5145: -7.21875
Feature 45145: -7.09375
Feature 2719: -7.09375
Feature 15415: -7.28125
Feature 24802: -7.15625
Feature 20375: -7.125
Feature 19505: -7.09375
Feature 38618: -7.25
Feature 36786: -7.125
Feature 47611: -7.03125
Feature 31726: -7.28125


KeyboardInterrupt: 

In [ ]:
check_feature(764, True)

message reduce behavior place se hello cel media club live line throw sport es door shape daughter run pu
 bottom mind mat polit year moment cut hour has sit comment drop un government d wait phone anticip south arm sport material place picture shape papel no en < choice viel fine arrived ii f third pu hang u er must just hello fig
mess rid comport lu sic infer cell media club abit punto get sport così porta col fig corr bel reg super m question polit dec momento tag ora aver sed comment g gi governo pap asp tele asp sud es sport mater lu f col cart no met rela sc pi bene arriv io cin ter bel append fu dire do app infer figura


<jax.Array(-5.9375, dtype=bfloat16)>

In [20]:
class FeatureSearch:
    def __init__(self, task, target_layer, early_stopping_steps=50, iterations=2000, seed=9, l1_coeff=1e-2, lr=3e-2, init_w=0.1, sae_v=6):
        self.task = task
        self.target_layer = target_layer
        self.sae = get_sae(target_layer, sae_v)
        self.seed = seed
        self.early_stopping_steps = early_stopping_steps
        self.iterations = iterations
        self.l1_coeff = l1_coeff
        self.lr = lr
        self.init_w = init_w

        pairs = tasks[task]
        pairs = [list(x) for x in pairs.items()]
        
        self.train_dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=self.seed)
        self.eval_dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=self.seed+1)

        self.eval_inputs = self.prepare_inputs(self.eval_dataset)
        self.train_inputs = self.prepare_inputs(self.train_dataset)

        self.initial_resids = self.get_initial_resids(self.train_inputs[0])

        self.lwg = jax.value_and_grad(self.get_loss, has_aux=True)
        self.taker = self.make_taker()
    
    def get_initial_resids(self, inputs):
        get_resids_initial = make_get_resids(llama, self.target_layer)
        get_resids_initial = jittify(get_resids_initial)

        initial_resids = get_resids_initial(inputs)
        return initial_resids

    def make_taker(self):
        taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
            lambda i, x: x if i >= self.target_layer else pz.nn.Identity()
        ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                        .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

        return taker

    def prepare_inputs(self, dataset: ICLDataset):
        tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
        inputs = tokenized_to_inputs(
            **tokenized
        )

        target_tokens = [x[1] for x in tokenizer(dataset.completions)["input_ids"]]
        target_tokens = jnp.asarray(target_tokens)

        return inputs, target_tokens        
    
    def get_loss(self, weights):
        weights = jax.nn.relu(weights)

        recon = jnp.einsum("fv,f->v", self.sae["W_dec"], weights)
        recon = recon.astype('bfloat16')

        modified = pz.nx.nmap(lambda a, b: a.at[-1].add(b))(
            self.initial_resids.untag("seq", "embedding"), pz.nx.wrap(recon, "embedding").untag("embedding")
            ).tag("seq", "embedding")

        inputs = dataclasses.replace(self.train_inputs[0], tokens=modified)

        logits = self.taker(inputs).unwrap("batch", "seq", "vocabulary")

        logprob_diff = get_logprob_diff(logits, self.eval_inputs[1])
        loss = -logprob_diff.mean()

        return loss + self.l1_coeff * jnp.linalg.norm(weights, ord=1), (int((weights != 0).sum()), loss)

    def train_step(self, weights, opt_state, optimizer):
        (loss, (l0, loss_)), grad = self.lwg(weights)

        updates, opt_state = optimizer.update(grad, opt_state, weights)
        weights = optax.apply_updates(weights, updates)

        return loss, weights, opt_state, dict(l0=l0, loss=loss_)

    def create_optimizer(self):
        optimizer = optax.chain(
            optax.adam(self.lr),
            optax.zero_nans(),
        )

        return optimizer

    def find_weights(self):
        weights = jnp.ones(self.sae["W_dec"].shape[0]) * self.init_w
        optimizer = self.create_optimizer()
        opt_state = optimizer.init(weights)

        min_loss = 1e9
        early_stopping_counter = 0

        for _ in (bar := trange(self.iterations)):
            loss, weights, opt_state, metrics = self.train_step(weights, opt_state, optimizer)

            if metrics["loss"] < min_loss:
                min_loss = metrics["loss"]
                early_stopping_counter = 0

            tk = jax.lax.top_k(weights, 2)

            bar.set_postfix(loss_optim=loss, **metrics, top=tk[1][0], top_diff=(tk[0][0] - tk[0][1]) / tk[0][0])

            early_stopping_counter += 1
            if early_stopping_counter > self.early_stopping_steps:
                break

        return weights, metrics
    
    def check_feature(self, feature, scale):
        steering_vector = self.sae["W_dec"][feature] * scale
        steering_vector =  steering_vector.astype('bfloat16')

        act_add = add_vector(
            llama, steering_vector, self.target_layer, scale=1.0, position="last"
        )

        logits = act_add(self.eval_inputs[0]).unwrap("batch", "seq", "vocabulary")


        logprobs = jax.nn.log_softmax(logits, axis=-1)
        answer_logprobs = logprobs[:, -1]

        target_logprobs = jnp.take_along_axis(answer_logprobs, self.eval_inputs[1][:, None], axis=-1).squeeze()

        return (target_logprobs - answer_logprobs.max(axis=-1)).mean()

    def check_features(self, features, scale):
        losses = jnp.hstack([self.check_feature(feature, scale) for feature in tqdm(features)])

        return features[losses.argmax()], losses.max(), losses.mean(), losses

    

In [17]:
info = """algo_max 18 0
algo_min 17 0
algo_last 18 -0.0116577
algo_first 18 0
algo_sum 20 0
algo_most_common 18 -0.00585938
"""

In [18]:
tasks_to_check = [x.split()[:2] for x in info.split("\n") if x]
tasks_to_check = [(x[0], int(x[1])) for x in tasks_to_check]
# tasks_to_check = [
#     ("en_es", 18),
#     ("en_it", 18),
#     ("en_fr", 18),
#     ("en_de", 18),
#     ("en_ru", 18)
# ]
tasks_to_check

[('algo_max', 18),
 ('algo_min', 17),
 ('algo_last', 18),
 ('algo_first', 18),
 ('algo_sum', 20),
 ('algo_most_common', 18)]

In [21]:
results = {}

for task, target_layer in tqdm(tasks_to_check):
    if target_layer == 21:
        target_layer = 20
    if target_layer == 14:
        target_layer = 12
    fs = FeatureSearch(task, target_layer, early_stopping_steps=200, sae_v=6)
    weights, metrics = fs.find_weights()
    i = jax.lax.top_k(jnp.abs(weights), 20)[1]
    best_feature, best_loss, mean_loss, losses = fs.check_features(i, scale=20)

    metrics = {k:float(v) for k, v in metrics.items()}

    results[task] = (best_feature, best_loss, mean_loss, losses, metrics)
    print(
        f"Task {task}, best feature {best_feature}, best loss {best_loss}, mean loss {mean_loss}, metrics {metrics}"
    )

  0%|          | 0/6 [00:00<?, ?it/s]

--2024-05-22 14:56:19--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l18-test-run-6-1.01E-05/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.125, 108.156.211.51, 108.156.211.95, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.125|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/f057cb46f3d871ba03c66e707e3b3d8299322f36fa433862dc3fdca956715614?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716648979&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjY0ODk3OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvZjA1N2NiND

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [33]:
results

{'location_continent': (<jax.Array(47188, dtype=int32)>,
  <jax.Array(-3.59375, dtype=bfloat16)>,
  <jax.Array(-4.5625, dtype=bfloat16)>,
  # jax.Array bfloat16(20,) ≈-4.6 ±0.42 [≥-5.1, ≤-3.6] nonzero:20
    Array([-4.90625, -4.59375, -4.8125, -4.375, -4.53125, -3.6875, -3.59375,
           -4.46875, -3.8125, -4.53125, -4.25, -4.875, -4.59375, -4.8125,
           -4.84375, -5.03125, -4.8125, -5.0625, -4.8125, -4.8125],      dtype=bfloat16)
  ,
  {'l0': 14.0, 'loss': 2.3125}),
 'football_player_position': (<jax.Array(26021, dtype=int32)>,
  <jax.Array(-6.90625, dtype=bfloat16)>,
  <jax.Array(-8.125, dtype=bfloat16)>,
  # jax.Array bfloat16(20,) ≈-8.1 ±0.47 [≥-8.7, ≤-6.9] nonzero:20
    Array([-8.3125, -8.6875, -8.5625, -8.5625, -8.5625, -8.4375, -8.375,
           -8.125, -8.4375, -7.6875, -6.90625, -8.125, -7.71875, -7.78125,
           -8.1875, -7.5, -7.40625, -8.5, -8.5, -8.3125], dtype=bfloat16)
  ,
  {'l0': 17.0, 'loss': 4.875})}

In [ ]:
import pickle

with open("results.pkl", "wb") as f:
    pickle.dump(results, f)
# with open("results.json", "w") as f:
#     json.dump(results, f)

In [ ]:
for task, (best_feature, best_loss, mean_loss, losses, metrics) in results.items():
    print(
        f"{task}, best feature {best_feature}, loss ratio {mean_loss / best_loss}"
    )

location_continent, best feature 47188, loss ratio 1.25
football_player_position, best feature 26021, loss ratio 1.17969
location_religion, best feature 27071, loss ratio 1.10156
location_language, best feature 177, loss ratio 2.21875
person_profession, best feature 48700, loss ratio 1.14844
location_country, best feature 10728, loss ratio 1.24219
country_capital, best feature 32281, loss ratio 1.57812
person_language, best feature 9753, loss ratio 3.20312
singular_plural, best feature 32291, loss ratio 1.19531
present_simple_past_simple, best feature 17167, loss ratio 1.75
antonyms, best feature 40483, loss ratio 1.51562
plural_singular, best feature 15284, loss ratio 2.03125
present_simple_past_perfect, best feature 23282, loss ratio 1.41406
present_simple_gerund, best feature 49149, loss ratio 2.07812
en_it, best feature 46157, loss ratio 1.14062
it_en, best feature 27586, loss ratio 1.39062
en_fr, best feature 20311, loss ratio 1.1875
en_es, best feature 20311, loss ratio 2.73438
f

In [ ]:
fs = FeatureSearch("en_de", 17)

--2024-05-22 00:38:44--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l17-test-run-6-4.52E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.90, 108.156.211.95, 108.156.211.125, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/1623d8da38be3171fcc8516a4cbe9fdb80e3d77e370aa5690895697649d688f3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716597524&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjU5NzUyNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvMTYyM2Q4ZGE

In [20]:
fs = FeatureSearch("en_ru", 18, early_stopping_steps=100, seed=10)
w = fs.find_weights()

--2024-05-22 01:20:39--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l18-test-run-6-1.01E-05/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.51, 108.156.211.90, 108.156.211.95, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.51|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/f057cb46f3d871ba03c66e707e3b3d8299322f36fa433862dc3fdca956715614?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716600039&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjYwMDAzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvZjA1N2NiNDZm

  0%|          | 0/2000 [00:00<?, ?it/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f12cc5bf290>>
Traceback (most recent call last):
  File "/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.12/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [21]:
jax.lax.top_k(w[0], 20)

[# jax.Array float32(20,) ≈8.6 ±1.1e+01 [≥-0.055, ≤3.1e+01] nonzero:20
   Array([ 3.09979153e+01,  2.82771473e+01,  2.81605225e+01,  2.23171616e+01,
           1.70592213e+01,  1.41996422e+01,  1.22735662e+01,  1.18328285e+01,
           3.67472243e+00,  2.47404289e+00,  1.58222604e+00, -2.69895289e-02,
          -2.70592347e-02, -2.96332240e-02, -3.23766917e-02, -4.17804755e-02,
          -4.86377254e-02, -5.33655584e-02, -5.33791259e-02, -5.49723543e-02],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥6_262, ≤46_627] nonzero:20
   Array([10615, 41937, 10151, 13937, 16245, 46627,  6262, 16804,  8286,
          25117, 10897, 28142, 21109, 21014, 44885, 43632, 44120,  7628,
          25342, 36901], dtype=int32)
 ]

In [22]:
fs.check_features(jax.lax.top_k(w[0], 20)[1], scale=20)

  0%|          | 0/20 [00:00<?, ?it/s]

(<jax.Array(16245, dtype=int32)>,
 <jax.Array(-13.125, dtype=bfloat16)>,
 <jax.Array(-13.8125, dtype=bfloat16)>,
 # jax.Array bfloat16(20,) ≈-1.4e+01 ±0.32 [≥-1.4e+01, ≤-1.3e+01] nonzero:20
   Array([-13.4375, -14.0625, -13.625, -13.75, -13.125, -13.4375, -13.8125,
          -13.625, -13.25, -13.9375, -13.375, -14, -13.875, -14.25, -14, -14,
          -14.1875, -13.8125, -14.1875, -14.0625], dtype=bfloat16)
 )

In [ ]:
fs.check_features(jax.lax.top_k(w[0], 20)[1], scale=20)

  0%|          | 0/20 [00:00<?, ?it/s]

(<jax.Array(41239, dtype=int32)>,
 <jax.Array(-4.21875, dtype=bfloat16)>,
 <jax.Array(-5.34375, dtype=bfloat16)>,
 # jax.Array bfloat16(20,) ≈-5.3 ±0.37 [≥-6.0, ≤-4.2] nonzero:20
   Array([-6, -5.28125, -5.65625, -5.15625, -5.03125, -5.15625, -5.53125,
          -5.25, -5.28125, -5.65625, -5.1875, -5.71875, -5.59375, -5.0625,
          -4.21875, -5.40625, -5.0625, -5.53125, -5.21875, -5.65625],      dtype=bfloat16)
 )

In [ ]:
jax.lax.top_k(w, 20)

[# jax.Array float32(20,) ≈5.4 ±2.9 [≥-0.014, ≤1.1e+01] nonzero:20
   Array([11.022304  ,  9.960602  ,  9.723273  ,  7.446827  ,  7.3656707 ,
           6.9718046 ,  6.634519  ,  6.3051505 ,  6.268656  ,  6.1738625 ,
           5.3788037 ,  4.8796234 ,  4.4844356 ,  4.196526  ,  3.5582683 ,
           2.7230783 ,  1.9743043 ,  1.7151848 ,  1.7031406 , -0.01442439],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥3_151, ≤47_066] nonzero:20
   Array([ 9644, 20311, 46157, 42589, 25117, 38261,  9337, 47066, 10897,
          30124,  3151,  3398, 46627, 36916, 17215, 10069, 29942, 39270,
           4999, 31578], dtype=int32)
 ]

In [ ]:
def find_features(task, target_layer):
    scale = 20
    early_stopping_steps = 50
    init_w = 0.1
    iterations = 2000

    sae = get_sae(target_layer, 6)
    dictionary = sae["W_dec"]

    pairs = tasks[task]
    pairs = [list(x) for x in pairs.items()]
    seed = 0

    dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=seed+1)


    tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
    inputs = tokenized_to_inputs(
        **tokenized
    )

    target_tokens = [x[1] for x in tokenizer(dataset.completions)["input_ids"]]
    target_tokens = jnp.asarray(target_tokens)

    jittify = lambda x: partial(jax.jit(lambda lr, *args, **kwargs: lr(*args, **kwargs)[1][0].value), x)
    get_resids_initial = make_get_resids(llama, target_layer)
    get_resids_initial = jittify(get_resids_initial)

    intital_resids = get_resids_initial(inputs)

    taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
        lambda i, x: x if i >= target_layer else pz.nn.Identity()
    ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                    .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

    optimizer = optax.chain(
        optax.adam(3e-2),
        optax.zero_nans(),    
    )

    lwg = jax.value_and_grad(get_loss, has_aux=True)
    

    weights = jnp.ones(dictionary.shape[0]) * init_w
    opt_state = optimizer.init(weights)

    min_loss = 1e9
    early_stopping_counter = 0

    for _ in (bar := trange(iterations)):
        loss, weights, opt_state, metrics = train_step(weights, opt_state, dictionary, inputs, target_tokens, taker, intital_resids, optimizer, lwg)

        if metrics["loss"] < min_loss:
            min_loss = metrics["loss"]
            early_stopping_counter = 0
        

        tk = jax.lax.top_k(weights, 2)

        bar.set_postfix(loss_optim=loss, **metrics, top=tk[1][0], top_diff=(tk[0][0] - tk[0][1]) / tk[0][0])

        early_stopping_counter += 1
        if early_stopping_counter > early_stopping_steps:
            break

    
    return jax.lax.top_k(jnp.abs(weights), 20)


In [ ]:
find_features(
    "en_es", 18
)

  0%|          | 0/2000 [00:00<?, ?it/s]

[# jax.Array float32(20,) ≈3.3 ±4.8 [≥0.42, ≤1.6e+01] nonzero:20
   Array([16.070333  , 12.78331   , 10.591674  , 10.247323  ,  5.344763  ,
           2.8526528 ,  1.5353229 ,  0.5156362 ,  0.50892174,  0.45389727,
           0.45202777,  0.44730943,  0.44149902,  0.43828565,  0.4329766 ,
           0.43184933,  0.43073234,  0.4306618 ,  0.42252657,  0.4161794 ],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥2_719, ≤48_025] nonzero:20
   Array([10018, 20311,  5145, 40023, 45145,  2719, 15415, 24802, 20375,
          19505, 38618, 36786, 47611, 31726, 12632, 10615, 47480, 48025,
           3048, 40850], dtype=int32)
 ]

In [ ]:
sae = get_sae(target_layer, 6)

In [ ]:
check_feature(20311, "en_es", 18, sae)

<jax.Array(-2.375, dtype=bfloat16)>